In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomContrast, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l1, l2
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import datetime

2023-12-08 15:44:44.692927: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512_VNNI
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-08 15:44:45.099122: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
# Set variables
epoch = 20
batch_size = 16
target_size = (299, 299)

# Load and preprocess images
def load_and_preprocess_images(data_df, image_folder, target_size):
    X = []
    y = []

    for index, row in data_df.iterrows():
        img_filename = row['image_path'][1:]  # Remove leading "/"
        img_path = os.path.join(image_folder, img_filename)
        label = row['label'] - 1

        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, target_size)
        img = img / 255.0

        X.append(img)
        y.append(label)

    return np.array(X), np.array(y)

In [4]:
# Load train data
train_df = pd.read_csv("train_images.csv")
script_dir = os.getcwd()
train_images = os.path.join(script_dir, 'train_images')

X_train, y_train = load_and_preprocess_images(train_df, train_images, target_size)

# Shuffle the data
X_train, y_train = shuffle(X_train, y_train, random_state=42)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [5]:
# Model architecture
base_model = tf.keras.applications.xception.Xception(weights="imagenet", include_top=False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
dropout = Dropout(0.5)(avg) 
output = tf.keras.layers.Dense(200, activation="softmax")(dropout)
model = tf.keras.Model(inputs=base_model.input, outputs=output)

for layer in base_model.layers:
    layer.trainable = False
    
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

# Implement early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

2023-12-08 15:46:18.927934: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512_VNNI
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-08 15:46:19.441676: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 18209 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB MIG 3g.20gb, pci bus id: 0000:32:00.0, compute capability: 8.0


In [6]:
# Train the model with early stopping
model.fit(
    X_train,
    y_train,
    epochs=epoch,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping] 
)

Epoch 1/20


2023-12-08 15:46:34.260783: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2023-12-08 15:46:37.512234: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-12-08 15:46:37.532664: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x265129b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-08 15:46:37.532695: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA A100-SXM4-40GB MIG 3g.20gb, Compute Capability 8.0
2023-12-08 15:46:37.560280: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


 1/99 [..............................] - ETA: 13:53 - loss: 5.4000 - accuracy: 0.0000e+00

2023-12-08 15:46:37.849943: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


99/99 [==============================] - 20s 116ms/step - loss: 4.3191 - accuracy: 0.1691 - val_loss: 3.2647 - val_accuracy: 0.3257
Epoch 2/20
99/99 [==============================] - 8s 80ms/step - loss: 2.6462 - accuracy: 0.4522 - val_loss: 2.5450 - val_accuracy: 0.4300
Epoch 3/20
99/99 [==============================] - 8s 81ms/step - loss: 2.0050 - accuracy: 0.5799 - val_loss: 2.2115 - val_accuracy: 0.4809
Epoch 4/20
99/99 [==============================] - 8s 80ms/step - loss: 1.6280 - accuracy: 0.6669 - val_loss: 2.0168 - val_accuracy: 0.5089
Epoch 5/20
99/99 [==============================] - 8s 81ms/step - loss: 1.3804 - accuracy: 0.7290 - val_loss: 1.8813 - val_accuracy: 0.5331
Epoch 6/20
99/99 [==============================] - 8s 80ms/step - loss: 1.2020 - accuracy: 0.7634 - val_loss: 1.7887 - val_accuracy: 0.5496
Epoch 7/20
99/99 [==============================] - 8s 81ms/step - loss: 1.0612 - accuracy: 0.7959 - val_loss: 1.7123 - val_accuracy: 0.5573
Epoch 8/20
99/99 [====

In [7]:
# Load test data paths
test_df = pd.read_csv("test_images_path.csv")
test_images = os.path.join(script_dir, 'test_images')
X_test, _ = load_and_preprocess_images(test_df, test_images, target_size)

In [8]:
# Make predictions
predictions = model.predict(X_test)

# Convert predictions to class labels
predicted_labels = np.argmax(predictions, axis=1) + 1

# Update the 'label' column in the test dataframe with the predicted labels
test_df['label'] = predicted_labels

selected_columns = ['id', 'label']
test_df_selected = test_df[selected_columns]

# Generate a timestamp for the filename
current_datetime = datetime.datetime.now()
timestamp = current_datetime.strftime("%Y%m%d_%H%M")
file_name = f"submission_{timestamp}.csv"

# Define the folder for submissions
submissions_folder = os.path.join(script_dir, 'submissions')

# Specify the full file path
file_path = os.path.join(submissions_folder, file_name)

# Save the final submission file
test_df_selected.to_csv(file_path, index=False)


125/125 [==============================] - 9s 62ms/step
